In [ ]:
# TODO:
# 
#https://www.youtube.com/watch?v=k1SzvvFtl4w
#https://www.youtube.com/watch?v=wypVcNIH6D4
#https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751
# https://realpython.com/python-keras-text-classification/
# https://radimrehurek.com/gensim/models/word2vec.html



In [3]:

from os import path
from nltk.stem.lancaster import LancasterStemmer
from random import choice, randint
import numpy as np 
import tensorflow as tf
import nltk, pickle, json, re, string, tflearn, warnings
warnings.filterwarnings("ignore")


In [ ]:
with open('cs_prompts.json') as file:
    data = json.load(file)

In [ ]:
# Preprocess and format the training data
def preprocess_train_data(data):
    stemmer = LancasterStemmer()

    words = []
    labels = list(data.keys())
    docs_x = []
    docs_y = []

    for label in labels:
        for pattern in data[label]['patterns']:
            tokens = nltk.word_tokenize(pattern)
            words.extend(tokens)
            docs_x.append(tokens)
            docs_y.append(label)

    # Pass over punctuation tokens
    ignored_tokens = [',', '.', '?', '!']
    words = [stemmer.stem(w.lower()) for w in words if w not in ignored_tokens]

    words = sorted(set(words))
    labels = sorted(labels)

    training = []
    output = []

    # Template for the BOW
    out_empty = list(np.zeros(len(labels)))

    for x, doc in enumerate(docs_x):
        bag = []
        stemmed = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in stemmed:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)  

    training = np.array(training)
    output = np.array(output)    
    
    return words, labels, training, output

In [ ]:
# If a pickle file of the processed training data exists, then it will be loaded
# Otherwise, the training data will be processed and saved in a pickle file

if path.exists('./data.pickle'):
    words, labels, training, output = preprocess_train_data(data)
    with open('data.pickle', 'wb') as file:
        pickle.dump((words, labels, training, output), file)
else:
    with open('data.pickle', 'rb') as file:
        words, labels, training, output = pickle.load(file)

In [ ]:
# Run this cell to create and train a new model
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')


In [ ]:
# Run this cell to load a previously trained model

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.load('model.tflearn')


In [ ]:
def bag_of_words(s, words, stemmer):
    bag = list(np.zeros(len(words)))
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i]=1
    return np.array(bag)

In [ ]:
def clear(): 
    from os import system, name
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
    # for mac and linux 
    else: 
        _ = system('clear') 

In [ ]:
def chat():

    clear()
    greetings = ["Hello! How can I help you today?", "Hello! What do you need help with today?", "Hi there, how can I help?"]
    print(choice(greetings))

    stemmer = LancasterStemmer()
    user_first_name = None
    user_last_name = None
    user_account_number = None

    def filter_punctuation(s):
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        return regex.sub('', s)

    while True:
        prediction = None

        inp = filter_punctuation(input("You: "))
        print(inp)
        if inp.lower() in ['quit', 'exit', 'stop']:
            break

        results = model.predict([bag_of_words(inp, words, stemmer)])
        
        result_index = np.argmax(results)
        
        result_max = np.max(results)

        if result_max > 0.66:

            prediction = labels[result_index]

            responses = data[prediction]['responses']
                    
            response = choice(responses)

            # print(prediction)

            print('Bot: ', choice(responses))

            if prediction in ['open_account', 'close_account', 'transfer_funds']:
                
                print("Bot: Please give me your first and last name.")

                inp = input("You: ")

                user_first_name, user_last_name = inp.split()

                print(f'Bot: Thank you, {user_first_name}.')

                if prediction == 'open_account':

                    user_account_number = ''.join([str(randint(1,9)) for i in range(12)])

                    print(f'Bot: Here is your new account number: {user_account_number}')

                    print('Bot: Can I help with anything else?')

                else:
                    print('Please give me your account number.')

                    inp = input("You: ")

                    print(f'Bot: Thank you, {user_first_name}. I have your account number')

                    print('Bot: Your requested action has taken place.')

                    print('Bot: Can I help with anything else?')
                    
        if prediction is 'goodbye':
            break

        if prediction is None:
            print(f"Bot: I'm sorry, I didn't quite get that. Can you rephrase your question?")



In [ ]:
chat()